<a href="https://colab.research.google.com/github/Pheonix10101/PRCV_p_5/blob/main/Task1_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
"""
Project 5: Recognition using Deep Networks

Author: Samruddhi Raut

This file contain following tasks
Task 1
F:Read the network and run it on the test set
"""
import os
import torch
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image
from matplotlib import pyplot as plt
import CNN


In [ ]:
torch.manual_seed(42)#In order to make your code repeatable, set the random seed for the torch package

**Defining main function**

In [ ]:

def main():
    loaded_net = CNN.NeuralNetwork()
    loaded_net_state_dict = torch.load('/content/samruddhi_neural.pt')
    loaded_net.load_state_dict(loaded_net_state_dict)
  
    train_loader, test_loader = CNN.read_and_print(CNN.BATCH_SIZE_TRAIN, CNN.BATCH_SIZE_TEST,
                                                           False)
# check if the model read from file is correct
# set model to eval()
    loaded_net.eval()
    test_loss = 0
    correct = 0
    extracted_examples = []
    extracted_preds = []
    extracted_targets = []
   
    with torch.no_grad(): # disable gradient calculation is useful for inference, backward() will not be called in testing
        c = 0
        for data, target in test_loader:
            output = loaded_net(data)
            for i in range(CNN.BATCH_SIZE_TEST):
                if c >= 10:
                    break
                extracted_examples.append(data[i])
                extracted_preds.append(output[i])
                extracted_targets.append(target[i])
                c += 1
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)

    print('\nTest over entire test set: Avg.loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
# run the model through the first 10 examples
    examples = torch.stack(extracted_examples)
    preds = torch.stack(extracted_preds)
    targets = torch.stack(extracted_targets)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        output = loaded_net(examples)
        print("output: ", end = "")
        test_loss += F.cross_entropy(output, targets, reduction = 'sum').item()
        pred = output.data.max(1, keepdim = True)[1]
        correct += pred.eq(targets.data.view_as(pred)).sum()
    test_loss /= len(examples)

    print('\nTest over entire test set: Avg.loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(examples), 100. * correct / len(examples)))

    for i in range(len(examples)):
        print(f"\nExample {i + 1}: \nOutput values :", end = "")
        print(['%.2f' % t for t in preds[i]])
        print("Predicted label: %d" % [t for t in preds[i]].index(max(preds[i])))
        print("Correct label: %d" % targets[i].item())

    fig = plt.figure()
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.tight_layout()
        plt.imshow(examples[i][0], cmap = 'gray', interpolation = 'none')
        plt.title("Prediction: %d" % ([t for t in preds[i]].index(max(preds[i]))))
        plt.xticks([])
        plt.yticks([])
    fig.show()
if __name__ == "__main__":
    main()